In [16]:
import os
import ipywidgets as widgets
from ipywidgets import Layout
import zipfile as zf
import shutil
import SimpleITK as sitk
import math
from radiomics import featureextractor

In [3]:
dicom_name_filler = widgets.Text(
    value='',
    placeholder = 'Name of your comparessed DICOM folder, end with .zip',
    description = 'DICOM: ',
    disabled = False,
    layout = widgets.Layout(width='500px')
)
dicom_file_uploader = widgets.FileUpload(
    accept='.zip',  
    multiple=False
)

In [4]:
display(dicom_name_filler)

Text(value='', description='DICOM: ', layout=Layout(width='500px'), placeholder='Name of your comparessed DICO…

In [5]:
display(dicom_file_uploader)

FileUpload(value={}, accept='.zip', description='Upload')

In [18]:
modeling_features_dic = {
    'RV':{'mean':41.8, 'std':9.527281401800353, 'unit':'mm', 'type':'user_input'},
    'SPAP':{'mean':55.69090909090909, 'std':27.990175372001012, 'unit':'mmHg', 'type':'user_input'},
    'PA':{'mean':3.2348636363636363, 'std':0.6519024463727505, 'unit':'cm', 'type':'user_input'},
    'SA':{'name':'original_shape_SurfaceArea', 'mean':222388.73242990908, 'std':45253.3147554674, 'type':'mask_label'},
    'FLA':{'name':'original_shape_Flatness', 'mean':0.5319147852954547, 'std':0.05120626560217771, 'type':'mask_label'},
    'SPH':{'name':'original_shape_Sphericity', 'mean':0.4983680056863636, 'std':0.023067973310864284, 'type':'mask_label'},
    'MIN':{'name':'original_firstorder_Minimum', 'mean':-0.3196264678863637, 'std':0.33672656523502387, 'type':'pa'},
    'SGE':{'name':'original_gldm_SmallDependenceLowGrayLevelEmphasis', 'mean':0.08525043123181816, 'std':0.01827983682244386, 'type':'pa'},
    'BSY':{'name':'original_ngtdm_Busyness','mean':1683.6258909918547,'std':5023.316178714534,'type':'pa'},
    'MDR':{'name':'original_shape_Maximum2DDiameterRow','mean':208.04910237636366,'std':27.00383979187061,'type':'pa'},
    'RMS':{'name':'original_firstorder_RootMeanSquared','mean':0.5434065142772727,'std':0.08810365068921959,'type':'pa'}
}

# feature extraction
settings = {}
settings['binWidth'] = 70
settings['resampledPixelSpacing'] = [5,5,5]
settings['interpolator'] = sitk.sitkNearestNeighbor
settings['normalize'] = True
extractor = featureextractor.RadiomicsFeatureExtractor(**settings)
extractor.enableImageTypes(Original={})

# the function to compute final result using modeling_params
def joint_lr_model(dic):
    #P(PAH)=1/(EXP(-0.604*RV+0.11*MIN-0.16*SGL-0.226*BSY-1.897*SPAP-0.927*PA+0.748*MDR-0.614*SPH+0.306*FLA-0.036*SA-0.286*RMS+18.27)+1)
    odd = -0.604*dic['RV']+0.11*dic['MIN']-0.16*dic['SGE']-0.226*dic['BSY']-1.897*dic['SPAP']-0.927*dic['PA']+0.748*dic['MDR']-0.614*dic['SPH']+0.306*dic['FLA']-0.036*dic['SA']-0.286*dic['RMS']+18.27
    result = 1/(math.exp(odd)+1)
    return result

# normalize all features using modeling_features_dic before computing the final modeling result
def normalize_features(cur_features_dic, modeling_features_dic):
    for feature in cur_features_dic:
        tmp = (cur_features_dic[feature] - modeling_features_dic[feature]['mean'])/modeling_features_dic[feature]['std']
        cur_features_dic[feature] = tmp
    return cur_features_dic

In [10]:
ct_indices_boxes = []
for feature in modeling_features_dic:
    if modeling_features_dic[feature]['type'] == 'user_input':
        val = widgets.FloatText(description=feature+':', disabled=False)
        unit = widgets.Label(modeling_features_dic[feature]['unit'])
        box = widgets.HBox([val, unit])
        ct_indices_boxes.append(box)
for box in ct_indices_boxes:
    display(box)

In [19]:
action_button = widgets.Button(
    description='Compute Result',
    disabled=False,
    button_style='',
    icon='check'
)
result_text = widgets.Text(
    value='',
    placeholder = 'Computed Result Probability',
    description = 'Probability: ',
    disabled = False,
    layout = widgets.Layout(width='500px')
)

In [23]:
def on_button_click():
    # decompress uploaded zip file
    dicom_file_name = dicom_name_filler.value.split('.')[0]
    files = zf.ZipFile(dicom_file_name, 'r')
    files.extractall(dicom_file_name)
    files.close()

    # read dicom file and generate .nii.gz file using SimpleITK image reader
    reader = sitk.ImageSeriesReader()
    tmp_name = reader.GetGDCMSeriesFileNames(os.path.join(dicom_file_name, str(dicom_file_name)))
    reader.SetFileNames(tmp_name)
    image = reader.Execute()
    sitk.WriteImage(image, dicom_file_name+'.nii.gz')

    # decompress nii.gz file to nii file
    with gzip.open(dicom_file_name+'.nii.gz', 'rb') as f_in:
        with open(dicom_file_name+'.nii', 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
    # for demonstration for now, mask_file_1 and mask_file_2 are pre-uploaded files
    cur_features_dic = {}
    # get user input feature values
    for box in ct_indices_boxes:
        if box.children[0].value==0:
            print('Please fill in all CT indices.')
            break
        else:
            cur_features_dic[box.children[0].description[:-1]] = box.children[0].value
            
    
    # extract features from mask-label.nrrd file
    image_file = dicom_file_name+'.nii.gz'
    mask_file_1 = dicom_file_name+'_mask-label.nrrd'
    fv1 = extractor.execute(image_file, mask_file_1)
    for feature in modeling_features_dic:
        if modeling_features_dic[feature]['type'] == 'mask_label':
            cur_features_dic[feature] = fv1[modeling_features_dic[feature]['name']]
        

    # extract features from pa.nrrd file
    mask_file_2 = dicom_file_name+'_pa101.nrrd'
    fv2 = extractor.execute(image_file, mask_file_2)
    for feature in modeling_features_dic:
        if modeling_features_dic[feature]['type'] == 'pa':
            cur_features_dic[feature] = fv2[modeling_features_dic[feature]['name']]

    # normalize features and compute result
    cur_features_dic = normalize_features(cur_features_dic, modeling_features_dic)
    modeling_result = joint_lr_model(cur_features_dic)
    
    # show result in result_text
    result_text.value = modeling_result

In [14]:
display(action_button)

Button(description='Compute Result', icon='check', style=ButtonStyle())

In [20]:
display(result_text)

Text(value='', description='Probability: ', layout=Layout(width='500px'), placeholder='Computed Result Probabi…

In [24]:
action_button.on_click(on_button_click)